In [97]:
import pandas as pd


######################
###     Minbzk     ###
######################
# RESULTS_FILE = "evaluation_minbzk_no_requests_minbzk_BM25S.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_stem_stopwords_minbzk_BM25S.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_real_words_minbzk_BM25S.csv"

# RESULTS_FILE = "evaluation_minbzk_no_requests_minbzk_keywords_BM25S.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_stem_stopwords_minbzk_keywords_BM25S.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_real_words_minbzk_keywords_BM25S.csv"

# RESULTS_FILE = "evaluation_minbzk_no_requests_minbzk_paraphrase_BM25S.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_stem_stopwords_minbzk_paraphrase_BM25S.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_real_words_minbzk_paraphrase_BM25S.csv"

# RESULTS_FILE = "evaluation_minbzk_no_requests_minbzk_real_words_BM25S.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_stem_stopwords_minbzk_real_words_BM25S.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_real_words_minbzk_real_words_BM25S.csv"



# RESULTS_FILE = "evaluation_minbzk_no_requests_minbzk_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_stem_stopwords_minbzk_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_real_words_minbzk_bert-base-dutch-cased.csv"

# RESULTS_FILE = "evaluation_minbzk_no_requests_minbzk_keywords_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_stem_stopwords_minbzk_keywords_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_real_words_minbzk_keywords_bert-base-dutch-cased.csv"

# RESULTS_FILE = "evaluation_minbzk_no_requests_minbzk_paraphrase_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_stem_stopwords_minbzk_paraphrase_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_real_words_minbzk_paraphrase_bert-base-dutch-cased.csv"

# RESULTS_FILE = "evaluation_minbzk_no_requests_minbzk_real_words_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_stem_stopwords_minbzk_real_words_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_real_words_minbzk_real_words_bert-base-dutch-cased.csv"



# RESULTS_FILE = "evaluation_minbzk_no_requests_minbzk_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_stem_stopwords_minbzk_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_real_words_minbzk_all-MiniLM-L6-v2.csv"

# RESULTS_FILE = "evaluation_minbzk_no_requests_minbzk_keywords_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_stem_stopwords_minbzk_keywords_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_real_words_minbzk_keywords_all-MiniLM-L6-v2.csv"

# RESULTS_FILE = "evaluation_minbzk_no_requests_minbzk_paraphrase_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_stem_stopwords_minbzk_paraphrase_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_real_words_minbzk_paraphrase_all-MiniLM-L6-v2.csv"

# RESULTS_FILE = "evaluation_minbzk_no_requests_minbzk_real_words_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_stem_stopwords_minbzk_real_words_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minbzk_no_requests_real_words_minbzk_real_words_all-MiniLM-L6-v2.csv"


######################
###     Minaz      ###
######################
# RESULTS_FILE = "evaluation_minaz_no_requests_minaz_BM25S.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_stem_stopwords_minaz_BM25S.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_real_words_minaz_BM25S.csv"

# RESULTS_FILE = "evaluation_minaz_no_requests_minaz_keywords_BM25S.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_stem_stopwords_minaz_keywords_BM25S.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_real_words_minaz_keywords_BM25S.csv"

# RESULTS_FILE = "evaluation_minaz_no_requests_minaz_paraphrase_BM25S.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_stem_stopwords_minaz_paraphrase_BM25S.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_real_words_minaz_paraphrase_BM25S.csv"

# RESULTS_FILE = "evaluation_minaz_no_requests_minaz_real_words_BM25S.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_stem_stopwords_minaz_real_words_BM25S.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_real_words_minaz_real_words_BM25S.csv"



# RESULTS_FILE = "evaluation_minaz_no_requests_minaz_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_stem_stopwords_minaz_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_real_words_minaz_bert-base-dutch-cased.csv"

# RESULTS_FILE = "evaluation_minaz_no_requests_minaz_keywords_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_stem_stopwords_minaz_keywords_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_real_words_minaz_keywords_bert-base-dutch-cased.csv"

# RESULTS_FILE = "evaluation_minaz_no_requests_minaz_paraphrase_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_stem_stopwords_minaz_paraphrase_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_real_words_minaz_paraphrase_bert-base-dutch-cased.csv"

# RESULTS_FILE = "evaluation_minaz_no_requests_minaz_real_words_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_stem_stopwords_minaz_real_words_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_real_words_minaz_real_words_bert-base-dutch-cased.csv"



# RESULTS_FILE = "evaluation_minaz_no_requests_minaz_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_stem_stopwords_minaz_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_real_words_minaz_all-MiniLM-L6-v2.csv"

# RESULTS_FILE = "evaluation_minaz_no_requests_minaz_keywords_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_stem_stopwords_minaz_keywords_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_real_words_minaz_keywords_all-MiniLM-L6-v2.csv"

# RESULTS_FILE = "evaluation_minaz_no_requests_minaz_paraphrase_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_stem_stopwords_minaz_paraphrase_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_real_words_minaz_paraphrase_all-MiniLM-L6-v2.csv"

# RESULTS_FILE = "evaluation_minaz_no_requests_minaz_real_words_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_stem_stopwords_minaz_real_words_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minaz_no_requests_real_words_minaz_real_words_all-MiniLM-L6-v2.csv"


######################
###     Minfin     ###
######################
# RESULTS_FILE = "evaluation_minfin_no_requests_minfin_BM25S.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_stem_stopwords_minfin_BM25S.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_real_words_minfin_BM25S.csv"

# RESULTS_FILE = "evaluation_minfin_no_requests_minfin_keywords_BM25S.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_stem_stopwords_minfin_keywords_BM25S.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_real_words_minfin_keywords_BM25S.csv"

# RESULTS_FILE = "evaluation_minfin_no_requests_minfin_paraphrase_BM25S.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_stem_stopwords_minfin_paraphrase_BM25S.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_real_words_minfin_paraphrase_BM25S.csv"

# RESULTS_FILE = "evaluation_minfin_no_requests_minfin_real_words_BM25S.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_stem_stopwords_minfin_real_words_BM25S.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_real_words_minfin_real_words_BM25S.csv"



# RESULTS_FILE = "evaluation_minfin_no_requests_minfin_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_stem_stopwords_minfin_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_real_words_minfin_bert-base-dutch-cased.csv"

# RESULTS_FILE = "evaluation_minfin_no_requests_minfin_keywords_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_stem_stopwords_minfin_keywords_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_real_words_minfin_keywords_bert-base-dutch-cased.csv"

# RESULTS_FILE = "evaluation_minfin_no_requests_minfin_paraphrase_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_stem_stopwords_minfin_paraphrase_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_real_words_minfin_paraphrase_bert-base-dutch-cased.csv"

# RESULTS_FILE = "evaluation_minfin_no_requests_minfin_real_words_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_stem_stopwords_minfin_real_words_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_real_words_minfin_real_words_bert-base-dutch-cased.csv"



# RESULTS_FILE = "evaluation_minfin_no_requests_minfin_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_stem_stopwords_minfin_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_real_words_minfin_all-MiniLM-L6-v2.csv"

# RESULTS_FILE = "evaluation_minfin_no_requests_minfin_keywords_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_stem_stopwords_minfin_keywords_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_real_words_minfin_keywords_all-MiniLM-L6-v2.csv"

# RESULTS_FILE = "evaluation_minfin_no_requests_minfin_paraphrase_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_stem_stopwords_minfin_paraphrase_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_real_words_minfin_paraphrase_all-MiniLM-L6-v2.csv"

# RESULTS_FILE = "evaluation_minfin_no_requests_minfin_real_words_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_stem_stopwords_minfin_real_words_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minfin_no_requests_real_words_minfin_real_words_all-MiniLM-L6-v2.csv"


######################
###     Minbuza    ###
######################
RESULTS_FILE = "evaluation_minbuza_no_requests_minbuza_BM25S.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_stem_stopwords_minbuza_BM25S.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_real_words_minbuza_BM25S.csv"

# RESULTS_FILE = "evaluation_minbuza_no_requests_minbuza_keywords_BM25S.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_stem_stopwords_minbuza_keywords_BM25S.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_real_words_minbuza_keywords_BM25S.csv"

# RESULTS_FILE = "evaluation_minbuza_no_requests_minbuza_paraphrase_BM25S.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_stem_stopwords_minbuza_paraphrase_BM25S.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_real_words_minbuza_paraphrase_BM25S.csv"

# RESULTS_FILE = "evaluation_minbuza_no_requests_minbuza_real_words_BM25S.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_stem_stopwords_minbuza_real_words_BM25S.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_real_words_minbuza_real_words_BM25S.csv"



# RESULTS_FILE = "evaluation_minbuza_no_requests_minbuza_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_stem_stopwords_minbuza_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_real_words_minbuza_bert-base-dutch-cased.csv"

# RESULTS_FILE = "evaluation_minbuza_no_requests_minbuza_keywords_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_stem_stopwords_minbuza_keywords_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_real_words_minbuza_keywords_bert-base-dutch-cased.csv"

# RESULTS_FILE = "evaluation_minbuza_no_requests_minbuza_paraphrase_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_stem_stopwords_minbuza_paraphrase_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_real_words_minbuza_paraphrase_bert-base-dutch-cased.csv"

# RESULTS_FILE = "evaluation_minbuza_no_requests_minbuza_real_words_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_stem_stopwords_minbuza_real_words_bert-base-dutch-cased.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_real_words_minbuza_real_words_bert-base-dutch-cased.csv"



# RESULTS_FILE = "evaluation_minbuza_no_requests_minbuza_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_stem_stopwords_minbuza_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_real_words_minbuza_all-MiniLM-L6-v2.csv"

# RESULTS_FILE = "evaluation_minbuza_no_requests_minbuza_keywords_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_stem_stopwords_minbuza_keywords_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_real_words_minbuza_keywords_all-MiniLM-L6-v2.csv"

# RESULTS_FILE = "evaluation_minbuza_no_requests_minbuza_paraphrase_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_stem_stopwords_minbuza_paraphrase_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_real_words_minbuza_paraphrase_all-MiniLM-L6-v2.csv"

# RESULTS_FILE = "evaluation_minbuza_no_requests_minbuza_real_words_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_stem_stopwords_minbuza_real_words_all-MiniLM-L6-v2.csv"
# RESULTS_FILE = "evaluation_minbuza_no_requests_real_words_minbuza_real_words_all-MiniLM-L6-v2.csv"

def extract_text(input_string):
    import re
    # Use regex to find text after 'min' and before the first '_'
    match = re.search(r'min([^_]+)_', input_string)
    if match:
        return match.group(1)
    else:
        return None


EVALUATION_FOLDER = "final_evaluation_minbzk" if "minbzk" in RESULTS_FILE else "final_evaluation"
woo_data = pd.read_csv(f"./{EVALUATION_FOLDER}/results/{RESULTS_FILE}")
DOCS_FOLDER = "final_docs_minbzk" if "minbzk" in RESULTS_FILE else "final_docs"
base_data = pd.read_csv(f"./{DOCS_FOLDER}/min{extract_text(RESULTS_FILE)}_no_requests/woo_merged.csv.gz")

# See how many pages correspond to the dossier
dossier_count = base_data['dossier_id'].value_counts().to_dict()
dossier_dict = base_data.groupby('dossier_id')['document_id'].apply(lambda x: list(set(x))).to_dict()
print(base_data['document_id'].nunique())

470


In [33]:
def calculate_metrics(woo_data, n):
    ap=[]
    precision=[]
    recall=[]
    f1=[]
    resultsTrue = 0
    resultsFalse = 0
    resultsTrueDocument = 0
    resultsFalseDocument = 0

    for index, row in woo_data.iterrows():
        results = []
        # Get all the data in a list
        for i in range(1, n + 1):
            dossier_col_name = f'dossier#{i}'
            if dossier_col_name in row:
                results.append(row[dossier_col_name])

        # # Check results
        # resultsTrue += results.count(True)
        # resultsFalse += results.count(False)

        # correct_documents = dossier_dict[row['dossier_id']]

        # tempResultsTrueDocument = set()
        # tempResultsFalseDocument = set()
        # for i in row['retrieved_document_ids'].split(", ")[:n]:
        #     if i in correct_documents:
        #         tempResultsTrueDocument.add(i)
        #     else:
        #         tempResultsFalseDocument.add(i)
        # resultsTrueDocument += len(tempResultsTrueDocument)
        # resultsFalseDocument += len(tempResultsFalseDocument)

        # Calculate Precision
        precision_score = results.count(True) / len(results)
        precision.append(precision_score)

        # Calculate Recall
        total_pages_nr = dossier_count.get(row['dossier_id'], 0)
        # recall_score = results.count(True) / len(results)
        recall_score = results.count(True) / total_pages_nr
        recall.append(recall_score)

        # Calculate f1
        if precision_score + recall_score > 0:
            f1_score = 2 * (precision_score * recall_score) / (precision_score + recall_score)
        else:
            f1_score = 0
        f1.append(f1_score)

        # Calculate MAP1
        average_precision = 0
        for k, result in enumerate(results, 1):
            if not result:
                continue
            average_precision += results[:k].count(True) / k
        
        # average_precision /= total_pages_nr
        average_precision /= len(results)
        ap.append(average_precision)

    # print(f"for n = {n}")
    # print(f"True results: {resultsTrue}")
    # print(f"False results: {resultsFalse}")
    # print(f"True results document: {resultsTrueDocument}")
    # print(f"False results document: {resultsFalseDocument}")
    return sum(ap) / len(ap), sum(precision) / len(precision), sum(recall) / len(recall), sum(f1) / len(f1)

In [10]:
results = {
    "Metric": ["MAP", "Precision", "Recall", "F1 Score"]
}

for i in [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
    map_score1, precision, recall, f1 = calculate_metrics(i)
    results[i] = [map_score1, precision, recall, f1]

# Convert the results dictionary to a DataFrame
results_df = pd.DataFrame(results)

# Set the 'Metric' column as the index
results_df.set_index("Metric", inplace=True)

pd.set_option('display.max_colwidth', None)
# Display the DataFrame
results_df

,10,20,30,40,50,60,70,80,90,100
Metric,,,,,,,,,,
MAP,0.799365,0.703688,0.633239,0.575259,0.527900,0.491526,0.460780,0.436227,0.412696,0.388735
Precision,0.807692,0.711538,0.643590,0.585577,0.537692,0.500641,0.470330,0.445673,0.421795,0.397692
Recall,0.453841,0.583844,0.673109,0.719305,0.749601,0.774200,0.796506,0.815408,0.829604,0.839716
F1 Score,0.411102,0.453003,0.467971,0.460387,0.447589,0.436519,0.427062,0.418748,0.408671,0.396592


test

In [100]:
import os

# Directory containing the evaluation results
# EVALUATION_FOLDER = "final_evaluation_minbzk/results"
EVALUATION_FOLDER = "final_evaluation/results"

# List of CSV files
######################
###     Minbzk     ###
######################
# csv_files = [
#     "evaluation_minbzk_no_requests_minbzk_BM25S.csv",
#     "evaluation_minbzk_no_requests_stem_stopwords_minbzk_BM25S.csv",
#     "evaluation_minbzk_no_requests_real_words_minbzk_BM25S.csv",
#     "evaluation_minbzk_no_requests_minbzk_keywords_BM25S.csv",
#     "evaluation_minbzk_no_requests_stem_stopwords_minbzk_keywords_BM25S.csv",
#     "evaluation_minbzk_no_requests_real_words_minbzk_keywords_BM25S.csv",
#     "evaluation_minbzk_no_requests_minbzk_paraphrase_BM25S.csv",
#     "evaluation_minbzk_no_requests_stem_stopwords_minbzk_paraphrase_BM25S.csv",
#     "evaluation_minbzk_no_requests_real_words_minbzk_paraphrase_BM25S.csv",
#     "evaluation_minbzk_no_requests_minbzk_real_words_BM25S.csv",
#     "evaluation_minbzk_no_requests_stem_stopwords_minbzk_real_words_BM25S.csv",
#     "evaluation_minbzk_no_requests_real_words_minbzk_real_words_BM25S.csv"
# ]
# csv_files = [
#     "evaluation_minbzk_no_requests_minbzk_bert-base-dutch-cased.csv",
#     "evaluation_minbzk_no_requests_stem_stopwords_minbzk_bert-base-dutch-cased.csv",
#     "evaluation_minbzk_no_requests_real_words_minbzk_bert-base-dutch-cased.csv",
#     "evaluation_minbzk_no_requests_minbzk_keywords_bert-base-dutch-cased.csv",
#     "evaluation_minbzk_no_requests_stem_stopwords_minbzk_keywords_bert-base-dutch-cased.csv",
#     "evaluation_minbzk_no_requests_real_words_minbzk_keywords_bert-base-dutch-cased.csv",
#     "evaluation_minbzk_no_requests_minbzk_paraphrase_bert-base-dutch-cased.csv",
#     "evaluation_minbzk_no_requests_stem_stopwords_minbzk_paraphrase_bert-base-dutch-cased.csv",
#     "evaluation_minbzk_no_requests_real_words_minbzk_paraphrase_bert-base-dutch-cased.csv",
#     "evaluation_minbzk_no_requests_minbzk_real_words_bert-base-dutch-cased.csv",
#     "evaluation_minbzk_no_requests_stem_stopwords_minbzk_real_words_bert-base-dutch-cased.csv",
#     "evaluation_minbzk_no_requests_real_words_minbzk_real_words_bert-base-dutch-cased.csv"
# ]
# csv_files = [
#     "evaluation_minbzk_no_requests_minbzk_all-MiniLM-L6-v2.csv",
#     "evaluation_minbzk_no_requests_stem_stopwords_minbzk_all-MiniLM-L6-v2.csv",
#     "evaluation_minbzk_no_requests_real_words_minbzk_all-MiniLM-L6-v2.csv",
#     "evaluation_minbzk_no_requests_minbzk_keywords_all-MiniLM-L6-v2.csv",
#     "evaluation_minbzk_no_requests_stem_stopwords_minbzk_keywords_all-MiniLM-L6-v2.csv",
#     "evaluation_minbzk_no_requests_real_words_minbzk_keywords_all-MiniLM-L6-v2.csv",
#     "evaluation_minbzk_no_requests_minbzk_paraphrase_all-MiniLM-L6-v2.csv",
#     "evaluation_minbzk_no_requests_stem_stopwords_minbzk_paraphrase_all-MiniLM-L6-v2.csv",
#     "evaluation_minbzk_no_requests_real_words_minbzk_paraphrase_all-MiniLM-L6-v2.csv",
#     "evaluation_minbzk_no_requests_minbzk_real_words_all-MiniLM-L6-v2.csv",
#     "evaluation_minbzk_no_requests_stem_stopwords_minbzk_real_words_all-MiniLM-L6-v2.csv",
#     "evaluation_minbzk_no_requests_real_words_minbzk_real_words_all-MiniLM-L6-v2.csv"
# ]

######################
###     Minaz      ###
######################
# csv_files = [
#     "evaluation_minaz_no_requests_minaz_BM25S.csv",
#     "evaluation_minaz_no_requests_stem_stopwords_minaz_BM25S.csv",
#     "evaluation_minaz_no_requests_real_words_minaz_BM25S.csv",
#     "evaluation_minaz_no_requests_minaz_keywords_BM25S.csv",
#     "evaluation_minaz_no_requests_stem_stopwords_minaz_keywords_BM25S.csv",
#     "evaluation_minaz_no_requests_real_words_minaz_keywords_BM25S.csv",
#     "evaluation_minaz_no_requests_minaz_paraphrase_BM25S.csv",
#     "evaluation_minaz_no_requests_stem_stopwords_minaz_paraphrase_BM25S.csv",
#     "evaluation_minaz_no_requests_real_words_minaz_paraphrase_BM25S.csv",
#     "evaluation_minaz_no_requests_minaz_real_words_BM25S.csv",
#     "evaluation_minaz_no_requests_stem_stopwords_minaz_real_words_BM25S.csv",
#     "evaluation_minaz_no_requests_real_words_minaz_real_words_BM25S.csv"
# ]
# csv_files = [
#     "evaluation_minaz_no_requests_minaz_bert-base-dutch-cased.csv",
#     "evaluation_minaz_no_requests_stem_stopwords_minaz_bert-base-dutch-cased.csv",
#     "evaluation_minaz_no_requests_real_words_minaz_bert-base-dutch-cased.csv",
#     "evaluation_minaz_no_requests_minaz_keywords_bert-base-dutch-cased.csv",
#     "evaluation_minaz_no_requests_stem_stopwords_minaz_keywords_bert-base-dutch-cased.csv",
#     "evaluation_minaz_no_requests_real_words_minaz_keywords_bert-base-dutch-cased.csv",
#     "evaluation_minaz_no_requests_minaz_paraphrase_bert-base-dutch-cased.csv",
#     "evaluation_minaz_no_requests_stem_stopwords_minaz_paraphrase_bert-base-dutch-cased.csv",
#     "evaluation_minaz_no_requests_real_words_minaz_paraphrase_bert-base-dutch-cased.csv",
#     "evaluation_minaz_no_requests_minaz_real_words_bert-base-dutch-cased.csv",
#     "evaluation_minaz_no_requests_stem_stopwords_minaz_real_words_bert-base-dutch-cased.csv",
#     "evaluation_minaz_no_requests_real_words_minaz_real_words_bert-base-dutch-cased.csv"
# ]
# csv_files = [
#     "evaluation_minaz_no_requests_minaz_all-MiniLM-L6-v2.csv",
#     "evaluation_minaz_no_requests_stem_stopwords_minaz_all-MiniLM-L6-v2.csv",
#     "evaluation_minaz_no_requests_real_words_minaz_all-MiniLM-L6-v2.csv",
#     "evaluation_minaz_no_requests_minaz_keywords_all-MiniLM-L6-v2.csv",
#     "evaluation_minaz_no_requests_stem_stopwords_minaz_keywords_all-MiniLM-L6-v2.csv",
#     "evaluation_minaz_no_requests_real_words_minaz_keywords_all-MiniLM-L6-v2.csv",
#     "evaluation_minaz_no_requests_minaz_paraphrase_all-MiniLM-L6-v2.csv",
#     "evaluation_minaz_no_requests_stem_stopwords_minaz_paraphrase_all-MiniLM-L6-v2.csv",
#     "evaluation_minaz_no_requests_real_words_minaz_paraphrase_all-MiniLM-L6-v2.csv",
#     "evaluation_minaz_no_requests_minaz_real_words_all-MiniLM-L6-v2.csv",
#     "evaluation_minaz_no_requests_stem_stopwords_minaz_real_words_all-MiniLM-L6-v2.csv",
#     "evaluation_minaz_no_requests_real_words_minaz_real_words_all-MiniLM-L6-v2.csv"
# ]

######################
###     Minfin     ###
######################
# csv_files = [
#     "evaluation_minfin_no_requests_minfin_BM25S.csv",
#     "evaluation_minfin_no_requests_stem_stopwords_minfin_BM25S.csv",
#     "evaluation_minfin_no_requests_real_words_minfin_BM25S.csv",
#     "evaluation_minfin_no_requests_minfin_keywords_BM25S.csv",
#     "evaluation_minfin_no_requests_stem_stopwords_minfin_keywords_BM25S.csv",
#     "evaluation_minfin_no_requests_real_words_minfin_keywords_BM25S.csv",
#     "evaluation_minfin_no_requests_minfin_paraphrase_BM25S.csv",
#     "evaluation_minfin_no_requests_stem_stopwords_minfin_paraphrase_BM25S.csv",
#     "evaluation_minfin_no_requests_real_words_minfin_paraphrase_BM25S.csv",
#     "evaluation_minfin_no_requests_minfin_real_words_BM25S.csv",
#     "evaluation_minfin_no_requests_stem_stopwords_minfin_real_words_BM25S.csv",
#     "evaluation_minfin_no_requests_real_words_minfin_real_words_BM25S.csv"
# ]
# csv_files = [
#     "evaluation_minfin_no_requests_minfin_bert-base-dutch-cased.csv",
#     "evaluation_minfin_no_requests_stem_stopwords_minfin_bert-base-dutch-cased.csv",
#     "evaluation_minfin_no_requests_real_words_minfin_bert-base-dutch-cased.csv",
#     "evaluation_minfin_no_requests_minfin_keywords_bert-base-dutch-cased.csv",
#     "evaluation_minfin_no_requests_stem_stopwords_minfin_keywords_bert-base-dutch-cased.csv",
#     "evaluation_minfin_no_requests_real_words_minfin_keywords_bert-base-dutch-cased.csv",
#     "evaluation_minfin_no_requests_minfin_paraphrase_bert-base-dutch-cased.csv",
#     "evaluation_minfin_no_requests_stem_stopwords_minfin_paraphrase_bert-base-dutch-cased.csv",
#     "evaluation_minfin_no_requests_real_words_minfin_paraphrase_bert-base-dutch-cased.csv",
#     "evaluation_minfin_no_requests_minfin_real_words_bert-base-dutch-cased.csv",
#     "evaluation_minfin_no_requests_stem_stopwords_minfin_real_words_bert-base-dutch-cased.csv",
#     "evaluation_minfin_no_requests_real_words_minfin_real_words_bert-base-dutch-cased.csv"
# ]
# csv_files = [
#     "evaluation_minfin_no_requests_minfin_all-MiniLM-L6-v2.csv",
#     "evaluation_minfin_no_requests_stem_stopwords_minfin_all-MiniLM-L6-v2.csv",
#     "evaluation_minfin_no_requests_real_words_minfin_all-MiniLM-L6-v2.csv",
#     "evaluation_minfin_no_requests_minfin_keywords_all-MiniLM-L6-v2.csv",
#     "evaluation_minfin_no_requests_stem_stopwords_minfin_keywords_all-MiniLM-L6-v2.csv",
#     "evaluation_minfin_no_requests_real_words_minfin_keywords_all-MiniLM-L6-v2.csv",
#     "evaluation_minfin_no_requests_minfin_paraphrase_all-MiniLM-L6-v2.csv",
#     "evaluation_minfin_no_requests_stem_stopwords_minfin_paraphrase_all-MiniLM-L6-v2.csv",
#     "evaluation_minfin_no_requests_real_words_minfin_paraphrase_all-MiniLM-L6-v2.csv",
#     "evaluation_minfin_no_requests_minfin_real_words_all-MiniLM-L6-v2.csv",
#     "evaluation_minfin_no_requests_stem_stopwords_minfin_real_words_all-MiniLM-L6-v2.csv",
#     "evaluation_minfin_no_requests_real_words_minfin_real_words_all-MiniLM-L6-v2.csv"
# ]

######################
###     Minbuza    ###
######################
# csv_files = [
#     "evaluation_minbuza_no_requests_minbuza_BM25S.csv",
#     "evaluation_minbuza_no_requests_stem_stopwords_minbuza_BM25S.csv",
#     "evaluation_minbuza_no_requests_real_words_minbuza_BM25S.csv",
#     "evaluation_minbuza_no_requests_minbuza_keywords_BM25S.csv",
#     "evaluation_minbuza_no_requests_stem_stopwords_minbuza_keywords_BM25S.csv",
#     "evaluation_minbuza_no_requests_real_words_minbuza_keywords_BM25S.csv",
#     "evaluation_minbuza_no_requests_minbuza_paraphrase_BM25S.csv",
#     "evaluation_minbuza_no_requests_stem_stopwords_minbuza_paraphrase_BM25S.csv",
#     "evaluation_minbuza_no_requests_real_words_minbuza_paraphrase_BM25S.csv",
#     "evaluation_minbuza_no_requests_minbuza_real_words_BM25S.csv",
#     "evaluation_minbuza_no_requests_stem_stopwords_minbuza_real_words_BM25S.csv",
#     "evaluation_minbuza_no_requests_real_words_minbuza_real_words_BM25S.csv"
# ]
# csv_files = [
#     "evaluation_minbuza_no_requests_minbuza_bert-base-dutch-cased.csv",
#     "evaluation_minbuza_no_requests_stem_stopwords_minbuza_bert-base-dutch-cased.csv",
#     "evaluation_minbuza_no_requests_real_words_minbuza_bert-base-dutch-cased.csv",
#     "evaluation_minbuza_no_requests_minbuza_keywords_bert-base-dutch-cased.csv",
#     "evaluation_minbuza_no_requests_stem_stopwords_minbuza_keywords_bert-base-dutch-cased.csv",
#     "evaluation_minbuza_no_requests_real_words_minbuza_keywords_bert-base-dutch-cased.csv",
#     "evaluation_minbuza_no_requests_minbuza_paraphrase_bert-base-dutch-cased.csv",
#     "evaluation_minbuza_no_requests_stem_stopwords_minbuza_paraphrase_bert-base-dutch-cased.csv",
#     "evaluation_minbuza_no_requests_real_words_minbuza_paraphrase_bert-base-dutch-cased.csv",
#     "evaluation_minbuza_no_requests_minbuza_real_words_bert-base-dutch-cased.csv",
#     "evaluation_minbuza_no_requests_stem_stopwords_minbuza_real_words_bert-base-dutch-cased.csv",
#     "evaluation_minbuza_no_requests_real_words_minbuza_real_words_bert-base-dutch-cased.csv"
# ]
csv_files = [
    "evaluation_minbuza_no_requests_minbuza_all-MiniLM-L6-v2.csv",
    "evaluation_minbuza_no_requests_stem_stopwords_minbuza_all-MiniLM-L6-v2.csv",
    "evaluation_minbuza_no_requests_real_words_minbuza_all-MiniLM-L6-v2.csv",
    "evaluation_minbuza_no_requests_minbuza_keywords_all-MiniLM-L6-v2.csv",
    "evaluation_minbuza_no_requests_stem_stopwords_minbuza_keywords_all-MiniLM-L6-v2.csv",
    "evaluation_minbuza_no_requests_real_words_minbuza_keywords_all-MiniLM-L6-v2.csv",
    "evaluation_minbuza_no_requests_minbuza_paraphrase_all-MiniLM-L6-v2.csv",
    "evaluation_minbuza_no_requests_stem_stopwords_minbuza_paraphrase_all-MiniLM-L6-v2.csv",
    "evaluation_minbuza_no_requests_real_words_minbuza_paraphrase_all-MiniLM-L6-v2.csv",
    "evaluation_minbuza_no_requests_minbuza_real_words_all-MiniLM-L6-v2.csv",
    "evaluation_minbuza_no_requests_stem_stopwords_minbuza_real_words_all-MiniLM-L6-v2.csv",
    "evaluation_minbuza_no_requests_real_words_minbuza_real_words_all-MiniLM-L6-v2.csv"
]


labels = [
    "Query file: raw, Database: raw",
    "Query file: raw, Database: no stopwords",
    "Query file: raw, Database: real words",
    "Query file: keywords, Database: raw",
    "Query file: keywords, Database: no stopwords",
    "Query file: keywords, Database: real words",
    "Query file: paraphrase, Database: raw",
    "Query file: paraphrase, Database: no stopwords",
    "Query file: paraphrase, Database: real words",
    "Query file: real words, Database: raw",
    "Query file: real words, Database: no stopwords",
    "Query file: real words, Database: real words"
]

# Store the results for each CSV
results_dict = {}

# Loop through the CSV files and calculate metrics
for csv_file in csv_files:
    file_path = os.path.join(EVALUATION_FOLDER, csv_file)
    
    # Read the data
    woo_data = pd.read_csv(file_path)
    
    # Call the calculate_metrics() function for each dataset
    results = {}
    for i in [10, 20, 30, 40, 50, 60, 70, 80, 90, 100]:
        map_score, precision, recall, f1 = calculate_metrics(woo_data, i)
        results[i] = [map_score, precision, recall, f1]
    
    # Store the results in a DataFrame for each file
    results_df = pd.DataFrame(results, index=["MAP", "Precision", "Recall", "F1 Score"])
    
    # Add the DataFrame to the dictionary
    results_dict[csv_file] = results_df

import matplotlib.pyplot as plt
import matplotlib.cm as cm

# Create 4 subplots for each metric
metrics = ["MAP", "Precision", "Recall"]

cmap = cm.get_cmap('tab20', len(results_dict))  # 'tab20' can be changed to other colormaps if desired
# Iterate over each metric
for metric in metrics:
    plt.figure(figsize=(13, 6))  # Adjust the size if needed

    # Plot each CSV file's results for the current metric
    for index, (csv_file, df) in enumerate(results_dict.items()):
        # Extract the model name from the CSV filename
        # Example: extracting "BM25S" or "bert-base-dutch-cased"
        model_name = csv_file.split('_')[-1].replace('.csv', '')

        # Remove the common part from the filename for labeling
        label = labels[index]

        # Plot the results with the cleaned label
        plt.plot(df.columns, df.loc[metric], label=label, color=cmap(index))

    # Adding titles and labels with dynamic model name
    plt.title(f"{metric} for {model_name}, for Ministry of Foreign Affairs")
    plt.xlabel("Number of Results (n)")
    plt.ylabel(metric)

    # Move the legend outside of the plot
    plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))

    plt.grid(True)

    # Adjust layout so the plot doesn't get cut off
    plt.tight_layout(rect=[0, 0, 0.85, 1])

    # Save the plot as an image with a distinct name that includes the model name
    plt.savefig(f"final_graphs/Evaluation_{metric}_{model_name}_minbuza.png")

    # Close the plot to avoid display conflicts
    plt.close()


C:\Users\Nicky\AppData\Local\Temp\ipykernel_18316\1613341941.py:236: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  cmap = cm.get_cmap('tab20', len(results_dict))  # 'tab20' can be changed to other colormaps if desired


In [69]:
# Combine results into a single DataFrame for display
combined_results = pd.concat(results_dict.values(), keys=labels)

map_table = combined_results.loc[(slice(None), 'MAP'), :]
precision_table = combined_results.loc[(slice(None), 'Precision'), :]
recall_table = combined_results.loc[(slice(None), 'Recall'), :]
f1_score_table = combined_results.loc[(slice(None), 'F1 Score'), :]

map_table

,,10,20,30,40,50,60,70,80,90,100
"Query file: raw, Database: raw",MAP,0.111082,0.084300,0.071060,0.060789,0.053935,0.049325,0.046127,0.043386,0.041659,0.042496
"Query file: raw, Database: no stopwords",MAP,0.057120,0.040835,0.034611,0.029557,0.026252,0.023509,0.021274,0.019625,0.018286,0.017318
"Query file: raw, Database: real words",MAP,0.085524,0.061620,0.051700,0.045395,0.041275,0.037589,0.034618,0.032436,0.030677,0.029249
"Query file: keywords, Database: raw",MAP,0.092187,0.072650,0.062644,0.055788,0.050702,0.046746,0.043881,0.042188,0.041483,0.043041
"Query file: keywords, Database: no stopwords",MAP,0.048575,0.038617,0.032806,0.029128,0.026065,0.024066,0.022272,0.020768,0.019712,0.018497
"Query file: keywords, Database: real words",MAP,0.062020,0.048195,0.040904,0.036075,0.033065,0.030598,0.029163,0.027726,0.026507,0.025669
"Query file: paraphrase, Database: raw",MAP,0.111722,0.084570,0.071016,0.063790,0.058010,0.053696,0.050004,0.047178,0.045562,0.046943
"Query file: paraphrase, Database: no stopwords",MAP,0.066797,0.052725,0.043982,0.039162,0.035606,0.032488,0.029914,0.028080,0.026487,0.025095
"Query file: paraphrase, Database: real words",MAP,0.068743,0.051988,0.045248,0.040359,0.037177,0.034557,0.032799,0.030997,0.029776,0.028656
"Query file: real words, Database: raw",MAP,0.054813,0.038155,0.030751,0.026825,0.023291,0.021192,0.019523,0.018527,0.018194,0.019464


In [70]:
precision_table

,,10,20,30,40,50,60,70,80,90,100
"Query file: raw, Database: raw",Precision,0.142857,0.119643,0.107823,0.097959,0.090102,0.085289,0.080029,0.075957,0.071542,0.066276
"Query file: raw, Database: no stopwords",Precision,0.082653,0.067347,0.060374,0.055102,0.050918,0.047789,0.045190,0.043686,0.042177,0.041071
"Query file: raw, Database: real words",Precision,0.112755,0.090816,0.084014,0.078444,0.074388,0.069983,0.065889,0.063329,0.061621,0.059949
"Query file: keywords, Database: raw",Precision,0.121939,0.110714,0.103571,0.095408,0.089694,0.085119,0.082070,0.079719,0.076587,0.070153
"Query file: keywords, Database: no stopwords",Precision,0.070918,0.061990,0.057823,0.054082,0.050612,0.048724,0.046283,0.044069,0.042574,0.041071
"Query file: keywords, Database: real words",Precision,0.091327,0.079847,0.074830,0.069770,0.067959,0.064796,0.063703,0.061990,0.059921,0.058673
"Query file: paraphrase, Database: raw",Precision,0.156122,0.132143,0.117007,0.109439,0.102449,0.097449,0.092711,0.087755,0.082993,0.076224
"Query file: paraphrase, Database: no stopwords",Precision,0.093878,0.081633,0.070748,0.066454,0.063571,0.059864,0.056122,0.054145,0.052721,0.050816
"Query file: paraphrase, Database: real words",Precision,0.103571,0.089286,0.083503,0.078699,0.074388,0.070663,0.068586,0.065944,0.064229,0.062296
"Query file: real words, Database: raw",Precision,0.077551,0.063010,0.056122,0.053061,0.048367,0.046854,0.044679,0.042857,0.041043,0.037704


In [71]:
recall_table

,,10,20,30,40,50,60,70,80,90,100
"Query file: raw, Database: raw",Recall,0.021747,0.030903,0.038872,0.045959,0.052038,0.056597,0.059941,0.064942,0.067705,0.068656
"Query file: raw, Database: no stopwords",Recall,0.010145,0.014870,0.018933,0.021532,0.024090,0.026648,0.029405,0.031732,0.033740,0.035136
"Query file: raw, Database: real words",Recall,0.010237,0.016270,0.023737,0.029957,0.034623,0.038304,0.041393,0.044695,0.048407,0.051253
"Query file: keywords, Database: raw",Recall,0.016419,0.025496,0.034012,0.045023,0.051387,0.055360,0.060101,0.063806,0.067205,0.069208
"Query file: keywords, Database: no stopwords",Recall,0.006879,0.011054,0.014300,0.016433,0.019107,0.021524,0.023446,0.024554,0.025987,0.027897
"Query file: keywords, Database: real words",Recall,0.008879,0.016038,0.020681,0.024887,0.028985,0.032238,0.036281,0.039098,0.041898,0.043887
"Query file: paraphrase, Database: raw",Recall,0.027453,0.036494,0.043885,0.052257,0.060158,0.065576,0.069505,0.073261,0.076721,0.077518
"Query file: paraphrase, Database: no stopwords",Recall,0.010701,0.015949,0.018477,0.021597,0.025072,0.027310,0.028828,0.031104,0.033009,0.034619
"Query file: paraphrase, Database: real words",Recall,0.013745,0.020605,0.029189,0.033637,0.037782,0.043843,0.047193,0.051221,0.054833,0.056999
"Query file: real words, Database: raw",Recall,0.013771,0.020967,0.024573,0.027879,0.029823,0.031926,0.034175,0.036437,0.038163,0.038598
